### Pre-setup

In [ ]:
import django
django.setup()

In [9]:
%%bash 
date

Tue Oct  1 08:51:46 UTC 2019


### Testing serializers

In [26]:
from datetime import datetime
from pprint import pprint

from django.utils import timezone
from django.utils.six import BytesIO
from rest_framework.renderers import JSONRenderer
from rest_framework.parsers import JSONParser

from toys.models import Toy
from toys.serializers import ToySerializer

In [12]:
toy_release_date = timezone.make_aware(
    datetime.now(),
    timezone.get_current_timezone()
)

In [16]:
toy1 = Toy(
    name='Snoopy taling action fig',
    description='Snoopy speaks five langs',
    release_date=toy_release_date,
    toy_category='Action Fig',
    was_included_in_home=False,
)
toy2 = Toy(
    name='Barbie',
    description='Barbie barbie~',
    release_date=toy_release_date,
    toy_category='Dolls',
    was_included_in_home=True,
)

# toy1.save()
# toy2.save()

In [21]:
# SELECT * FROM toys_toy ORDER BY 'name';
print(
    toy1.pk, 
    toy1.description, 
    toy1.release_date,
    toy1.toy_category, 
    toy1.was_included_in_home,
)
print(
    toy2.pk, 
    toy2.description, 
    toy2.release_date,
    toy2.toy_category, 
    toy2.was_included_in_home,
)

5
Snoopy speaks five langs
2019-10-01 08:56:43.115166+00:00
Action Fig
False
6
Barbie barbie~
2019-10-01 08:56:43.115166+00:00
Dolls
True


In [31]:
# Raw data (hmm)

cereal_for_toy1 = ToySerializer(toy1)
cereal_for_toy2 = ToySerializer(toy2)

pprint(cereal_for_toy1.data); print()
pprint(cereal_for_toy2.data)

{'description': 'Snoopy speaks five langs',
 'name': 'Snoopy taling action fig',
 'pk': 5,
 'release_date': '2019-10-01T08:56:43.115166Z',
 'toy_category': 'Action Fig',
 'was_included_in_home': False}

{'description': 'Barbie barbie~',
 'name': 'Barbie',
 'pk': 6,
 'release_date': '2019-10-01T08:56:43.115166Z',
 'toy_category': 'Dolls',
 'was_included_in_home': True}


In [56]:
# JSONRenderer   Render `data` into JSON, returning a bytestring.
# JSONParser     Parses incoming bytestream as JSON, returns the resulting data.

# Basically,
#   render to JSON     ready to be sent       e.g. http POST
#   parser into obj    to native datatypes    e.g. save to database

In [37]:
# Render into JSON

json_renderer = JSONRenderer()
toy1_tojson = json_renderer.render(cereal_for_toy1.data)
toy2_tojson = json_renderer.render(cereal_for_toy2.data)

toy1_tojson
toy2_tojson

b'{"pk":5,"name":"Snoopy taling action fig","description":"Snoopy speaks five langs","release_date":"2019-10-01T08:56:43.115166Z","toy_category":"Action Fig","was_included_in_home":false}'

b'{"pk":6,"name":"Barbie","description":"Barbie barbie~","release_date":"2019-10-01T08:56:43.115166Z","toy_category":"Dolls","was_included_in_home":true}'

In [50]:
# From serialized data (JSON) to the an instance of Toy
# 1. deserailize
# 2. parse

json_str_for_new_toy = (
    '{'
    '"name":"cellphone", "description":"just a cellphone yo",'
    '"release_date": "2017-10-20T06:06:20.522532Z",'
    '"toy_category": "Electronics",'
    '"was_included_in_home":false'
    '}'
)
json_bytes_for_new_toy = bytes(json_str_for_new_toy, encoding='UTF-8')
stream_for_new_toy = BytesIO(json_bytes_for_new_toy)

parser = JSONParser()
parsed_new_toy = parser.parse(stream_for_new_toy)

parsed_new_toy

{'name': 'cellphone',
 'description': 'just a cellphone yo',
 'release_date': '2017-10-20T06:06:20.522532Z',
 'toy_category': 'Electronics',
 'was_included_in_home': False}

In [58]:
new_toy_serializer = ToySerializer(data=parsed_new_toy)

if new_toy_serializer.is_valid():
    # toy3 = new_toy_serializer.save()
    print(toy3.name)

cellphone
